### This code takes normalized and binned ChIP-seq data and runs it through Cyber-T statistical analysis.
#### The normal model of Cyber-T is not appropriate for count data, but coding this pipeline was proof-of-concept for a later project that I worked on. It also gave me practice in implementing a statistical modeling method as used in Cyber-T. The "binning" of ChIP-seq data refers to binning counts across standardized genomic windows with the intent of making differential "expression" analysis tractable. The implementation of such code is the subject of an unreleased publication.

In [1]:
#=============================================================
# 1. Load Data
#=============================================================

import pandas as pd

# Our data is contained in Norm_Reads
Norm_Reads = pd.read_csv("QNorm_Reads.txt",sep=",")
Norm_Reads = Norm_Reads.set_index("Window")

# We load raw reads so we have a reference for ordering the data frames
RawReads = pd.read_csv("RawAreas.txt",sep="\t")
RawReads = RawReads.set_index("Window")

#=============================================================
# 2. Statistical Analysis of Data
#=============================================================

# Cyber-T implementation
# 1. t-statistic: (mu1 - mu2) / SD_bayes
# 2. SD_Bayes <- sqrt((conf * SD_prior^2 + (N - 1) * SD_actual^2)/(conf + N - 2)), 
# where conf is a weight reflecting confidence in data, while SD_actual is the actual SD for the data line
# 3. SD_prior = Inferred SD value from a Lowess curve, given a mean
# The lowess curve is defined by the curve of best fit for a Mean vs SD plot, for all individual Mean vs SD values 

Bayes_Weighting = 10

import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
import scipy.stats as stats

Norm_Reads_PS = Norm_Reads.loc[:,["Norm_PS1","Norm_PS2"]]
Norm_Reads_PS["PS_Mean"] = Norm_Reads_PS[["Norm_PS1","Norm_PS2"]].sum(axis=1)/2.0
Norm_Reads_PS["PS_SD"] = np.sqrt((Norm_Reads_PS["Norm_PS1"] - Norm_Reads_PS["PS_Mean"])**2 +  \
                                 (Norm_Reads_PS["Norm_PS2"] - Norm_Reads_PS["PS_Mean"])**2)

Norm_Reads_OS = Norm_Reads.loc[:,["Norm_OS1","Norm_OS2"]]
Norm_Reads_OS["OS_Mean"] = Norm_Reads_OS[["Norm_OS1","Norm_OS2"]].sum(axis=1)/2.0
Norm_Reads_OS["OS_SD"] = np.sqrt((Norm_Reads_OS["Norm_OS1"] - Norm_Reads_OS["OS_Mean"])**2 +  \
                                 (Norm_Reads_OS["Norm_OS2"] - Norm_Reads_OS["OS_Mean"])**2)

# http://statsmodels.sourceforge.net/devel/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html

Norm_Reads_PS["PS_SD_Prior"] = lowess(endog = np.array(Norm_Reads_PS["PS_SD"]),exog = np.array(Norm_Reads_PS["PS_Mean"]),
                                      delta = 0.01 * (max(np.array(Norm_Reads_PS["PS_Mean"])) - min(np.array(Norm_Reads_PS["PS_Mean"]))),
                                      return_sorted = False)
Norm_Reads_PS["PS_SD_Bayes"] = np.sqrt((Bayes_Weighting*Norm_Reads_PS["PS_SD_Prior"]**2  +  (2 - 1)*Norm_Reads_PS["PS_SD"])
                                       /(Bayes_Weighting + 2 - 2))


Norm_Reads_OS["OS_SD_Prior"] = lowess(endog = np.array(Norm_Reads_OS["OS_SD"]),exog = np.array(Norm_Reads_OS["OS_Mean"]),
                                      delta = 0.01 * (max(np.array(Norm_Reads_OS["OS_Mean"])) - min(np.array(Norm_Reads_OS["OS_Mean"]))),
                                      return_sorted = False)
Norm_Reads_OS["OS_SD_Bayes"] = np.sqrt((Bayes_Weighting*Norm_Reads_OS["OS_SD_Prior"]**2  +  (2 - 1)*Norm_Reads_OS["OS_SD"]**2)
                                       /(Bayes_Weighting + 2 - 2))


Norm_Reads_PostStat = Norm_Reads_PS.join(Norm_Reads_OS)

# http://www.statsdirect.com/help/parametric_methods/utt.htm
# 2-sided t-test with UNEQUAL VARIANCES
Norm_Reads_PostStat["DF"] = ((Norm_Reads_PostStat["PS_SD_Bayes"]**2/2 + Norm_Reads_PostStat["OS_SD_Bayes"]**2/2)**2 \
                          / ((Norm_Reads_PostStat["PS_SD_Bayes"]**2/2)**2 + (Norm_Reads_PostStat["OS_SD_Bayes"]**2/2)**2))
Norm_Reads_PostStat["D_stat"] = ((Norm_Reads_PostStat["PS_Mean"] - Norm_Reads_PostStat["OS_Mean"])
                                 /np.sqrt(Norm_Reads_PostStat["PS_SD_Bayes"]**2/2 + Norm_Reads_PostStat["OS_SD_Bayes"]**2/2))
Norm_Reads_PostStat["pval"] = stats.t.sf(np.abs(Norm_Reads_PostStat["D_stat"]), Norm_Reads_PostStat["DF"])*2

#=============================================================
# 3. Sort/Organize Data
#=============================================================

# Split normalized reads by chromosome
Human_Chromosomes_2 = ["chrM","chr1'","chr2'","chr3","chr4","chr5","chr6","chr7","chr8","chr9",
                    "chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17",
                    "chr18","chr19","chr20","chr21","chr22","chrX","chrY"]
Areas_By_Chromosome = {}
Areas_By_Chromosome_Raw = {}

for i in Human_Chromosomes_2:
    Areas_By_Chromosome[i] = Norm_Reads_PostStat.ix[[j for j in Norm_Reads_PostStat.index if i in j]]
    Areas_By_Chromosome_Raw[i] = RawReads.ix[[j for j in RawReads.index if i in j]]
    Areas_By_Chromosome[i] = Areas_By_Chromosome[i].reindex(list(Areas_By_Chromosome_Raw[i].index.values))
    
Areas_By_Chromosome["chr1"] = Areas_By_Chromosome["chr1'"]
Areas_By_Chromosome["chr2"] = Areas_By_Chromosome["chr2'"]

C:\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [39]:
#=============================================================
# Organize Human Gene Annotations
#=============================================================

# hg19 annotations, downloaded 4/3/2016
# Gene annotations were previously consolidated in such a way that 
# 1. 5 kB regions upstream and downstream were included in coordinates, and 
# 2. overlapping genes form combined single entries

GeneRegions = open("GeneList_Consolidated.csv","r")
num_lines = sum(1 for line in GeneRegions)
GeneRegions.close()
GeneRegions = open("GeneList_Consolidated.csv","r")
Human_Chromosomes = ["chrM","chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9",
                    "chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17",
                    "chr18","chr19","chr20","chr21","chr22","chrX","chrY"]

# First row is a header row - get rid of it 
Region = GeneRegions.readline()

# Create a mapping between gene locations and the normalized chipseq data
# Regions_Of_Interest is a dict that contains the following information
# key: Entrez IDs of the region of interest
# value: a list of tuples (chromosome, window_begin, window_end) that correspond to the row labels 
# in the normalized chipseq data
Regions_Of_Interest = {}
for Lines in range(0,num_lines):
    Region = GeneRegions.readline().strip()
    Region = Region.split(",")
    if len(Region) < 2:
        break
    Region_Chromosome = Region[1]
    Region_Begin = int(Region[2])
    Region_End = int(Region[3])
    Region_Label = Region[6]
    if Region_Chromosome in Human_Chromosomes:
        Chrom_Regions = Areas_By_Chromosome[Region_Chromosome].index
    else:
        continue
    Region_Location = [i for i in Chrom_Regions 
                       if (int(i.split(", ")[1]) > Region_Begin and int(i.split(", ")[2].split(")")[0]) < Region_End) or
                      (int(i.split(", ")[2].split(")")[0]) > Region_Begin and int(i.split(", ")[2].split(")")[0]) < Region_End) 
                       or (int(i.split(", ")[1]) > Region_Begin and int(i.split(", ")[1]) < Region_End)]
    Regions_Of_Interest[Region_Label] = Region_Location
    
GeneRegions.close()

# Backup of the mapping
# ONLY ACTIVATE WHEN YOU'RE READY TO BACK SOMETHING UP
#import pickle

#pickle.dump(Regions_Of_Interest,open("RegionMapping_Backup.p","wb"))

In [2]:
#=============================================================
# Organize Human Gene Annotations, cont.
#=============================================================

# Reload mapping
import pickle

Regions_Of_Interest = pickle.load(open("RegionMapping_Backup.p","rb"))

In [3]:
#=============================================================
# DATA VISUALIZATION
#=============================================================

# Pick individual genes to visualize, by Entrez ID
# Entrez ID of KLF2 is 10365, for example. Entrez ID of KLF4 is 9314.
# Make a plot of selected data of interest (which also shows gene annotation superimposed over data)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

Desired_IDs = [3691]

for i in Desired_IDs:
    Search_For_ID = [elem for elem in Regions_Of_Interest.keys() if (str(i) == elem or str(i)+":" in elem or ":"+str(i) in elem)]
    if len(Search_For_ID) == 0:
        print i," not found"
        continue
    Gene_ID = Search_For_ID[0]
    
    Data_Of_Interest = Norm_Reads_PostStat.ix[Regions_Of_Interest[Gene_ID]]
    # 1. Extract chromosomal range and window size
    Chr_Range = list(Data_Of_Interest.index.values)
    Chromosome = eval(Chr_Range[0])[0]
    Window_Size = eval(Chr_Range[0])[2] - eval(Chr_Range[0])[1]
    Begin = eval(Chr_Range[0])[1]
    End = eval(Chr_Range[-1])[2]

    fig = plt.gca()
    plt.plot((Begin,End),(0,0),color="black",linewidth=1)
    
    if ":" in Gene_ID:
        Gene_ID = i

    # Plot data
    for i in list(Data_Of_Interest.index.values):
        Coords = eval(i)
        plt.plot((Coords[1], Coords[2]), (Data_Of_Interest.loc[i,"Norm_OS1"], Data_Of_Interest.loc[i,"Norm_OS1"]),
                 color = "#e74c3c",linewidth=4)
        plt.plot((Coords[1], Coords[2]), (Data_Of_Interest.loc[i,"Norm_OS2"], Data_Of_Interest.loc[i,"Norm_OS2"]),
                 color = '#7b241c',linewidth=4)
        plt.plot((Coords[1], Coords[2]), (Data_Of_Interest.loc[i,"Norm_PS1"], Data_Of_Interest.loc[i,"Norm_PS1"]),
                 color = '#3498db',linewidth=4)
        plt.plot((Coords[1], Coords[2]), (Data_Of_Interest.loc[i,"Norm_PS2"], Data_Of_Interest.loc[i,"Norm_PS2"]),
                 color = 'blue',linewidth=4)
        if float(Data_Of_Interest.loc[i,"pval"]) < 0.05:
            plt.text((Coords[1]+Coords[2])/2.0-50, max(Data_Of_Interest.loc[i,"Norm_OS1"],Data_Of_Interest.loc[i,"Norm_OS2"],
                                                       Data_Of_Interest.loc[i,"Norm_PS1"],Data_Of_Interest.loc[i,"Norm_PS2"])+10,"*",size=30)

    # Plot gene (green portion shows where the TSS is)
    Gene_Annotations = pd.read_csv("GeneIndex_Ordered.csv",sep=",",index_col=0)
    Plotted_Gene_Begin = int(Gene_Annotations.loc[Gene_Annotations["gene_id"] == int(Gene_ID),"start"])
    Plotted_Gene_End = int(Gene_Annotations.loc[Gene_Annotations["gene_id"] == int(Gene_ID),"end"])
    Plotted_Gene_Name = list(Gene_Annotations.loc[Gene_Annotations["gene_id"] == int(Gene_ID),"gene_name"])[0]
    plt.plot((Plotted_Gene_Begin+5000, Plotted_Gene_End-5000), (-1000, -1000),color = 'black',linewidth=6)

    if (Gene_Annotations.loc[Gene_Annotations["gene_id"] == int(Gene_ID),"strand"] == "+").any():
        plt.plot((Plotted_Gene_Begin+5000, Plotted_Gene_Begin+5010), (-1000, -1000),color = 'green',linewidth=6)
        plt.text(Plotted_Gene_Begin+5000,-750,Plotted_Gene_Name)
    else:
        plt.plot((Plotted_Gene_End-5010, Plotted_Gene_End-5000), (-1000, -1000),color = 'green',linewidth=6)
        plt.text(Plotted_Gene_Begin+5000,-1250,Plotted_Gene_Name)

    # Labels and formatting
    fig.get_xaxis().get_major_formatter().set_useOffset(False)
    fig.get_xaxis().get_major_formatter().set_scientific(False)
    fig.set_xlabel(Chromosome,size=18)
    fig.set_ylabel("Normalized area under curve per 1000-nt window",size=15)
    fig.tick_params(axis='both', which='major', labelsize=15)

    # Legend
    OS1_patch = mpatches.Patch(color='#e74c3c', label='OS1')
    OS2_patch = mpatches.Patch(color='#7b241c', label='OS2')
    PS1_patch = mpatches.Patch(color='#3498db', label='PS1')
    PS2_patch = mpatches.Patch(color='blue', label='PS2')
    TSS_patch = mpatches.Patch(color='green', label='TSS')
    plt.legend(handles=[OS1_patch,OS2_patch,PS1_patch,PS2_patch,TSS_patch])

    plt.show()
        

#Regions_Of_Interest["10365"]

#Data_Of_Interest = Norm_Reads_PostStat.ix[Regions_Of_Interest["10365"]]

#Data_Of_Interest.to_csv("KLF2.csv")

In [3]:
#=============================================================
# DATA VISUALIZATION - CHECKING LOWESS FUNCTION ON DATA
#=============================================================

NormPS_Sample = Norm_Reads_PS.sample(n=5000)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

NormPS_Sample.plot.scatter(x="PS_Mean",y="PS_SD",xlim=(0,3500),ylim=(0,3500))
NormPS_Sample.plot.scatter(x="PS_Mean",y="PS_SD_Prior",xlim=(0,3500),ylim=(0,3500))
plt.show()

NameError: name 'Norm_Reads_PS' is not defined

In [77]:
#=============================================================
# DATA FILTRATION
#=============================================================

# Find all genes with at least one region that is differentially acetylated.
Gene_Annotations = pd.read_csv("GeneIndex_Ordered.csv",sep=",",index_col=0)
All_IDs = list(Gene_Annotations["gene_id"])

Good_IDs = []
for i in All_IDs:
    Search_For_ID = [elem for elem in Regions_Of_Interest.keys() if (str(i) == elem or str(i)+":" in elem or ":"+str(i) in elem)]
    if len(Search_For_ID) == 0:
        print i," not found"
        continue
    Gene_ID = Search_For_ID[0]
    
    Data_Of_Interest = Norm_Reads_PostStat.ix[Regions_Of_Interest[Gene_ID]]
    
    if Data_Of_Interest.loc[Data_Of_Interest["pval"] < 0.05,"pval"].count() > 0:
        Good_IDs += [i]
    
Good_Gene_Information = Gene_Annotations.ix[Good_IDs]

Good_Gene_Information.to_csv("GenesWithDEAcetylation.csv")

441058  not found
727764  not found
7146  not found
389831  not found
403340  not found


In [9]:
import pandas as pd
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
import scipy.stats as stats
from copy import deepcopy
import ast
import math

# Generalized version of statistical analysis - debugged


def StatAnalysis_OfQNormBins(QNormArea_File, RawArea_File, QNormReadsWithPVals_File, Control_Sets, Experimental_Sets,
                             annotation_file, Bayes_Weighting=10):
    # =============================================================
    # 1. Load Data
    # ============================================================

    # Our data is contained in Norm_Reads
    Norm_Reads = pd.read_csv(QNormArea_File, sep=",")
    Norm_Reads = Norm_Reads.set_index("Window")

    # We load raw reads so we have a reference for ordering the data frames
    # Note 6/13/17: Norm_Reads are reordered in the normalization function, so this step is not necessary
    #RawReads = pd.read_csv(RawArea_File, sep="\t")
    #RawReads = RawReads.set_index("Window")

    # =============================================================
    # 2. Statistical Analysis of Data
    # =============================================================

    # Cyber-T implementation
    # According to Shakti:
    # 1. t-statistic: (mu1 - mu2) / SD_bayes
    # 2. SD_Bayes <- sqrt((conf * SD_prior^2 + (N - 1) * SD_actual^2)/(conf + N - 2)),
    # where conf is a weight reflecting confidence in data, while SD_actual is the actual SD for the data line
    # 3. SD_prior = Inferred SD value from a Lowess curve, given a mean
    # The lowess curve is defined by the curve of best fit for a Mean vs SD plot, for all individual Mean vs SD values

    # Bayes_Weighting = 10

    Norm_Reads_Control = Norm_Reads.loc[:, Control_Sets]
    Norm_Reads_Control["Ctrl_Mean"] = Norm_Reads_Control[Control_Sets].sum(axis=1) / float(len(Control_Sets))

    temp = deepcopy(Norm_Reads_Control)
    for column in Control_Sets:
        temp[column] = temp[column] - temp["Ctrl_Mean"]
    Norm_Reads_Control["Ctrl_SD"] = np.sqrt((temp[Control_Sets] ** 2).sum(axis=1))

    Norm_Reads_Experiment = Norm_Reads.loc[:, Experimental_Sets]
    Norm_Reads_Experiment["Expt_Mean"] = Norm_Reads_Experiment[Experimental_Sets].sum(axis=1) / float(len(Experimental_Sets))
    temp2 = deepcopy(Norm_Reads_Experiment)
    for column in Experimental_Sets:
        temp2[column] = temp2[column] - temp2["Expt_Mean"]
    Norm_Reads_Experiment["Expt_SD"] = np.sqrt((temp2[Experimental_Sets] ** 2).sum(axis=1))

    # http://statsmodels.sourceforge.net/devel/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html

    Norm_Reads_Control["Ctrl_SD_Prior"] = lowess(endog=np.array(Norm_Reads_Control["Ctrl_SD"]),
                                                 exog=np.array(Norm_Reads_Control["Ctrl_Mean"]),
                                                 delta=0.01 * (max(np.array(Norm_Reads_Control["Ctrl_Mean"])) - min(
                                                     np.array(Norm_Reads_Control["Ctrl_Mean"]))),
                                                 return_sorted=False)
    Norm_Reads_Control["Ctrl_SD_Bayes"] = np.sqrt(
        (Bayes_Weighting * Norm_Reads_Control["Ctrl_SD_Prior"] ** 2 + (len(Control_Sets) - 1) * Norm_Reads_Control["Ctrl_SD"] ** 2)
        / (Bayes_Weighting + len(Control_Sets) - 2.0))

    Norm_Reads_Experiment["Expt_SD_Prior"] = lowess(endog=np.array(Norm_Reads_Experiment["Expt_SD"]),
                                                    exog=np.array(Norm_Reads_Experiment["Expt_Mean"]),
                                                    delta=0.01 * (
                                                    max(np.array(Norm_Reads_Experiment["Expt_Mean"])) - min(
                                                        np.array(Norm_Reads_Experiment["Expt_Mean"]))),
                                                    return_sorted=False)
    Norm_Reads_Experiment["Expt_SD_Bayes"] = np.sqrt(
        (Bayes_Weighting * Norm_Reads_Experiment["Expt_SD_Prior"] ** 2 + (len(Experimental_Sets) - 1) *
         Norm_Reads_Experiment["Expt_SD"] ** 2)
        / (Bayes_Weighting + len(Experimental_Sets) - 2.0))

    Norm_Reads_PostStat = Norm_Reads_Control.join(Norm_Reads_Experiment)

    # http://www.statsdirect.com/help/parametric_methods/utt.htm
    # 2-sided t-test with UNEQUAL VARIANCES
    Norm_Reads_PostStat["DF"] = (
        ((Norm_Reads_PostStat["Ctrl_SD_Bayes"] ** 2 / float(len(Control_Sets))
          + Norm_Reads_PostStat["Expt_SD_Bayes"] ** 2 / float(len(Experimental_Sets))) ** 2)
        / (
            (((Norm_Reads_PostStat["Ctrl_SD_Bayes"] ** 2) / (float(len(Control_Sets)) ** 2)) / (
            float(len(Control_Sets) - 1.0)))
            + (((Norm_Reads_PostStat["Expt_SD_Bayes"] ** 2) / (float(len(Experimental_Sets)) ** 2)) / (
            float(len(Experimental_Sets) - 1.0)))
        )
    )

    Norm_Reads_PostStat["D_stat"] = ((Norm_Reads_PostStat["Ctrl_Mean"] - Norm_Reads_PostStat["Expt_Mean"])
                                     / np.sqrt(Norm_Reads_PostStat["Ctrl_SD_Bayes"] ** 2 / float(len(Control_Sets))
                                               + Norm_Reads_PostStat["Expt_SD_Bayes"] ** 2 / float(len(Experimental_Sets)))
                                     )

    Norm_Reads_PostStat["pval"] = stats.t.sf(np.abs(Norm_Reads_PostStat["D_stat"]), Norm_Reads_PostStat["DF"]) * 2

    Norm_Reads_PostStat["bonf"] = Norm_Reads_PostStat["pval"] * Norm_Reads_PostStat.shape[0]
    Norm_Reads_PostStat.loc[Norm_Reads_PostStat["bonf"] > 1,"bonf"] = 1

    # =============================================================
    # 3. Exporting And Binning Data
    # =============================================================

    # The analyzed data file will be saved.
    # Data binned by chromosome will also be saved, for quick plotting access.

    Norm_Reads_PostStat.to_csv(QNormReadsWithPVals_File)

    Columns_Of_Importance = Norm_Reads.columns.tolist() + ["pval","bonf"]
    Condensed_NormReads = Norm_Reads_PostStat[Columns_Of_Importance]
    condensed_filename = QNormReadsWithPVals_File.split(".txt")[0] + "_condensed.csv"

    Chromosomes = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,"X","Y"]

    for i in Chromosomes:
        StringSearch = "('chr" + str(i) + "',"
        Chrom_Windows = [j for j in list(Condensed_NormReads.index) if StringSearch in j]
        if len(Chrom_Windows) > 0:
            Chromosome_Data = Condensed_NormReads.ix[Chrom_Windows]
            chrom_filename = QNormReadsWithPVals_File.split(".txt")[0] + "_chr" + str(i) + "_condensed.csv"
            Chromosome_Data.to_csv(chrom_filename)

    Window = ast.literal_eval(Chrom_Windows[0])
    WindowSize = Window[2] - Window[1]


    Condensed_NormReads.to_csv(condensed_filename)
    
    return Condensed_NormReads, WindowSize
    
def Testing_GeneFile_Generation(annotation_file, Condensed_NormReads, Control_Sets, Experimental_Sets,QNormReadsWithPVals_File,
                               WindowSize):

    # =============================================================
    # 4. Finding all differentially modified gene regions
    # =============================================================

    # Possible ways to summarize a gene, given multi-window data:
    # (1) Min p-value, with corresponding LFC
    # (2) Max LFC, with corresponding p-value
    # Only (1) will be implemented for now.

    Annotation_File = annotation_file.set_index("Gene stable ID")
    List_Of_Annotated_Genes = list(Annotation_File.index)
    List_Of_Annotated_Genes = [x.encode('UTF8') for x in List_Of_Annotated_Genes]

    GeneData = []

    for Ensembl_ID in List_Of_Annotated_Genes:
        Chromosome = Annotation_File.loc[Ensembl_ID, "Chromosome/scaffold name"]
        GeneBegin = Annotation_File.loc[Ensembl_ID, "Gene start (bp)"]
        GeneEnd = Annotation_File.loc[Ensembl_ID, "Gene end (bp)"]
        Strand = Annotation_File.loc[Ensembl_ID, "Strand"]
        Name = Annotation_File.loc[Ensembl_ID, "Gene name"]
        
        Chromosomes = ["1","2","3","4","5","6","7","8","9","10","11","12",
                       "13","14","15","16","17","18","19","20","21","22","X","Y"]
        
        if str(Chromosome) not in Chromosomes:
            continue
        else:
            GeneWindowBegin = GeneBegin - 5000
            GeneWindowEnd = GeneEnd + 5000

            # From gene annotation data, define the regions of interest in (chromosome, begin, end) format
            DataFromGeneWindowBegin = int(math.floor(GeneWindowBegin / float(WindowSize)) * float(WindowSize))
            DataFromGeneWindowEnd = int(math.ceil(GeneWindowEnd / float(WindowSize)) * float(WindowSize))
            Regions_Of_Interest = []
            i = DataFromGeneWindowBegin
            while i < DataFromGeneWindowEnd:
                Region = "('chr" + str(Chromosome) + "', " + str(i) + ", " + str(i + WindowSize) + ")"
                Regions_Of_Interest += [Region]
                i += WindowSize

            Data_Of_Interest = Condensed_NormReads.ix[Regions_Of_Interest]
            Min_PVal = Data_Of_Interest["bonf"].min()
            #print Min_PVal
            #print Annotation_File.loc[Ensembl_ID,:]
        
            Control_Sum = Data_Of_Interest.loc[Data_Of_Interest["bonf"] == Min_PVal,:]
        
            # Account for case where gene has multiple min p-vals of same value 
            # (usual case: gene is not DE and has p values of 1 across all windows)
            # The fold change to be reported for this case will be the fold change of largest magnitude
            if isinstance(Control_Sum, pd.DataFrame):
                Control_Sum = Data_Of_Interest.loc[Data_Of_Interest["bonf"] == Min_PVal,Control_Sets].sum(axis=1) + 1.0
                Expt_Sum = Data_Of_Interest.loc[Data_Of_Interest["bonf"] == Min_PVal,Experimental_Sets].sum(axis=1) + 1.0
                LogFoldChanges = np.log2(Expt_Sum/Control_Sum)
                LargestFoldChangeLocation = LogFoldChanges.abs().idxmax()
                LogFoldChange = LogFoldChanges[LargestFoldChangeLocation]
            else:
                Control_Sum = sum(Data_Of_Interest.loc[Data_Of_Interest["bonf"] == Min_PVal,Control_Sets].tolist()) + 1.0
                Expt_Sum = sum(Data_Of_Interest.loc[Data_Of_Interest["bonf"] == Min_PVal,Experimental_Sets].tolist()) + 1.0
                FoldChange = Control_Sum / Expt_Sum
                LogFoldChange = np.log2(FoldChange)

            DataRow = [Ensembl_ID, Name,Chromosome,LogFoldChange,Min_PVal]
            GeneData += [DataRow]

    Final_GeneData = pd.DataFrame(GeneData,
                                  columns=["Ensembl ID","Name","Chromosome","Most significant log2FC","Adj. P Value"])

    genedata_filename = QNormReadsWithPVals_File.split(".txt")[0] + "_genesummary.csv"
    Final_GeneData.to_csv(genedata_filename)

In [26]:
Annotation_File = pd.read_csv("Human GRCh37.p13 07.19.17.csv")
#Annotation_File = Annotation_File.set_index("Gene stable ID")
Annotation_File.head()

,Gene stable ID,HGNC symbol,Gene start (bp),Gene end (bp),Strand,Chromosome/scaffold name,Gene name
0,ENSG00000223972,DDX11L1,11869,14412,1,1,DDX11L1
1,ENSG00000227232,WASH7P,14363,29806,-1,1,WASH7P
2,ENSG00000243485,MIR1302-10,29554,31109,1,1,MIR1302-10
3,ENSG00000237613,FAM138A,34554,36081,-1,1,FAM138A
4,ENSG00000268020,OR4G4P,52473,54936,1,1,OR4G4P


In [37]:
Norm_Reads_PostStat[["D_stat","pval"]].head()

,D_stat,pval
Window,,
"('chr19', 52903000, 52904000)",5.339418,0.091882
"('chr8', 36265000, 36266000)",0.411270,0.722079
"('chr7', 25503000, 25504000)",-4.208745,0.085011
"('chr3', 20704000, 20705000)",3.377282,0.113791
"('chr6', 165394000, 165395000)",-2.029002,0.189813


In [56]:
Test_General_Code.loc[RawReads.index,["D_stat","pval"]].head()

,D_stat,pval
Window,,
"('chrM', 0, 1000)",4.141372,0.000035
"('chrM', 1000, 2000)",3.854526,0.000116
"('chrM', 2000, 3000)",3.936609,0.000083
"('chrM', 3000, 4000)",4.746932,0.000002
"('chrM', 4000, 5000)",4.077633,0.000045
